In [ ]:
~from google.colab import drive
drive.mount('/content/drive')

default_path = '/content/drive/My Drive'

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/메타버스 아카데미/본과정/Python/data/dataset
%ls

/content/drive/MyDrive/메타버스 아카데미/본과정/Python/data/dataset
0/     1/     2/     3/     4/     5/     dataset/
0.csv  1.csv  2.csv  3.csv  4.csv  5.csv  wandb/


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=27d58bf9f6296789c7b74d0764642864459ba510b528664f7638b5727e7cc3ae
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.8 MB/s eta 0:00:00


In [ ]:
import os
import sys
import copy
import uuid

import torch
import torch.nn as nn
from torchvision import models, transforms

import cv2
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm


def set_augmentations(task):
    if task == 'train':
        temp_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

    elif task in ['valid', 'test']:
        temp_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
    else:
        print('#' * 30, 'Please confirm your task name', '#' * 30)

    return temp_transforms

def visualize_augmentations(dataset, index=0, samples=20, columns=5):
    dataset = copy.deepcopy(dataset)
    dataset.transforms = A.Compose([transform for transform in dataset.transforms if not isinstance(transform, (A.Normalize, ToTensorV2))])

    rows = samples // columns
    _, axis = plt.subplots(nrows=rows, ncols=columns, figsize=(12, 6))

    for i in range(samples):
        image, _ = dataset[index]
        axis.ravel()[i].imshow(image)
        # image = image.permute(1, 2, 0)
        # print('#' * 50, image.shape)
        # https://stackoverflow.com/questions/50963283/imshow-doesnt-need-convert-from-bgr-to-rgb
        # https://github.com/philipperemy/keract/issues/46
        axis.ravel()[i].set_axis_off()

    plt.tight_layout()
    plt.show()


def train(train_loader, valid_loader, epochs, model, device, criterion, optimizer, scheduler):
    import wandb

    wandb.login()
    wandb.init(project=f"{model._get_name()}")
    wandb.config = {"learning_rate": optimizer.param_groups[0]['lr'],
                    "epochs": epochs,
                    "batch_size": train_loader.batch_size,
                    "classes": 6,}
    wandb.watch(model)

    model_name = model._get_name()
    best_valid_accuracy = 0.0

    train_steps = len(train_loader)

    save_path = os.path.join('./', '{}_best.pt'.format(model_name))

    accuracy_df = pd.DataFrame(index=list(range(epochs)), columns=['Epoch',
                                                                      'Model',
                                                                      'Optimizer',
                                                                      'Batch size',
                                                                      'Scheduler',
                                                                      'Learning Rate',
                                                                      'Train Loss',
                                                                      'Train Accuracy',
                                                                      'Valid Loss',
                                                                      'Valid Accuracy'])

    if os.path.exists(save_path):
        best_valid_accuracy = max(pd.read_csv('./{}_accuracy.csv'.format(model_name))['Accuracy'].tolist())

    for epoch in range(1, epochs + 1):
        train_running_loss = 0.0
        train_corrects = 0
        train_length = 0

        model.train()
        train_bar = tqdm(train_loader, file=sys.stdout, colour='green')

        for image_batch, label_batch in train_bar:
            image_batch, label_batch = image_batch.to(device), label_batch.to(device)

            outputs = model(image_batch)
            train_loss = criterion(outputs, label_batch)

            optimizer.zero_grad()
            train_corrects += (torch.argmax(outputs, dim=1) == label_batch).sum().item()
            train_loss.backward()
            optimizer.step()
            train_running_loss += train_loss.item()
            train_length += image_batch.size(0)

            train_bar.desc = 'Train Epoch: [{:3d} / {:3d}]   Loss: {:.3f}'.format(
                epoch, epochs, train_loss.data
            )

        train_accuracy = train_corrects / train_length
        train_average_loss = train_running_loss / train_steps

        valid_accuracy, valid_average_loss = validate(valid_loader, model, device, criterion, epoch)

        accuracy_df.loc[epoch, 'Epoch'] = epoch
        accuracy_df.loc[epoch, 'Model'] = model_name
        accuracy_df.loc[epoch, 'Optimizer'] = type(optimizer).__name__
        accuracy_df.loc[epoch, 'Batch size'] = train_loader.batch_size
        accuracy_df.loc[epoch, 'Learning Rate'] = optimizer.param_groups[0]['lr']
        accuracy_df.loc[epoch, 'Train Loss'] = round(train_average_loss, 3)
        accuracy_df.loc[epoch, 'Train Accuracy'] = round(train_accuracy, 3)
        accuracy_df.loc[epoch, 'Valid Loss'] = round(valid_average_loss, 3)
        accuracy_df.loc[epoch, 'Valid Accuracy'] = round(valid_accuracy, 3)

        wandb.log({
            "Epoch": epoch,
            "Learning Rate": optimizer.param_groups[0]['lr'],
            "Train Loss": round(train_average_loss, 3),
            "Train Accuracy": round(train_accuracy, 3),
            "Valid Loss": round(valid_average_loss, 3),
            "Valid Accuracy": round(valid_accuracy, 3),
        })

        if scheduler is not None:
            accuracy_df.loc[epoch, 'Scheduler'] = type(scheduler).__name__
            scheduler.step()

        print('Epoch: [{:3d} / {:3d}]   Train Loss: {:.3f}   Train Accuracy: {:.3f}   Valid Loss: {:.3f}   Valid Accuracy: {:.3f}'.format(
            epoch, epochs, train_average_loss, train_accuracy, valid_average_loss, valid_accuracy
        ))

        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            torch.save(model.state_dict(), save_path)

        if epoch == epochs:
            accuracy_df.to_csv('./{}_accuracy.csv'.format(model_name), index=False)

        if epoch % 10 == 0:
            torch.save(model.state_dict(), './{}_{}_epoch.pt'.format(model_name, epoch))


def validate(valid_loader, model, device, criterion, epoch):
    valid_steps = len(valid_loader)
    valid_running_loss = 0.0
    valid_corrects = 0
    valid_length = 0

    model.eval()
    with torch.no_grad():

        valid_bar = tqdm(valid_loader, file=sys.stdout, colour='red')

        for valid_batch in valid_bar:
            image_batch, label_batch = valid_batch
            image_batch, label_batch = image_batch.to(device), label_batch.to(device)

            outputs = model(image_batch)
            valid_loss = criterion(outputs, label_batch)
            valid_corrects += (torch.argmax(outputs, dim=1) == label_batch).sum().item()
            valid_running_loss += valid_loss.item()
            valid_length += image_batch.size(0)

            collect_failed_predictions(epoch, image_batch, label_batch, (torch.argmax(outputs, dim=1) == label_batch))

    valid_accuracy = valid_corrects / valid_length
    valid_average_loss = valid_running_loss / valid_steps

    return valid_accuracy, valid_average_loss


def collect_failed_predictions(epoch, image_batch, label_batch, corrects_tensor):
    dir_path = os.path.join('./', 'incorrect', str(epoch))
    os.makedirs(dir_path, exist_ok=True)

    incorrect_list = corrects_tensor.tolist()
    label_list = label_batch.tolist()

    for index, correct in enumerate(incorrect_list):
        if not correct:
            uuid_path = '{}_{}.png'.format(label_list[index], str(uuid.uuid1()))
            dest_path = os.path.join(dir_path, uuid_path)
            image_tensor = image_batch[index]
            image = get_image_from_tensor(image_tensor)

            cv2.imwrite(dest_path, image)


def denormalize(normalized_image):
    # # In case: normalize by /255.
    # image = image * 255.0

    IMAGENET_MEAN, IMAGENET_STD = np.array([0.485, 0.456, 0.406]), np.array([0.229, 0.224, 0.225])
    image = np.clip(255.0 * (normalized_image * IMAGENET_STD + IMAGENET_MEAN), 0, 255)

    return image


def get_image_from_tensor(image_tensor):
    # Tensor formed image [C, H, W]
    # tensor -> numpy
    temp_image = image_tensor.detach().cpu().numpy()
    # [C,H,W] -> [H,W,C]
    temp_image = np.transpose(temp_image, (1, 2, 0))
    # Denormalize
    temp_image = denormalize(temp_image)
    # np.float32 -> np.uint8
    image = temp_image.astype(np.uint8)
    # # BGR to RGB
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return image


def test(test_loader, device, label_volume):
    model = models.mobilenet_v2(pretrained=True)
    model.classifier[1].out_features = label_volume
    model.load_state_dict(torch.load(f'./{model._get_name()}/{model._get_name()}_best.pt'.format(), map_location=device))
    model.to(device)

    correct = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for i, (image, label) in enumerate(test_loader):
            images, labels = image.to(device), label.to(device)

            output = model(images)
            _, argmax = torch.max(output, dim=1)
            total += images.size(0)
            correct += (labels == argmax).sum().item()

        accuracy = accuracy_function(correct, total)
        print('Accuracy: ', accuracy)


def accuracy_function(correct, total):
    accuracy = correct / total * 100
    return accuracy


# import os
# from custom_dataset import CustomDataset
# data_path = os.path.join('../', 'dataset')
# transforms = set_augmentations('train')
# train_dataset = CustomDataset(data_path, transforms=transforms, task='train')
# visualize_augmentations(train_dataset)

In [ ]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import models, datasets

from timm.loss import LabelSmoothingCrossEntropy

# Set Paramerts
epoch_num = 50
batch_size = 32
label_volume = 6
dataset_path = os.path.join('../', 'dataset')
step_size = 9                                 # resnet50
gamma = 0.1                                   # resnet50
learning_rate = 2e-4                          # resnet50-AdamW
# step_size = 30                                # resnet18
# learning_rate = 1e-3                          # resnet18-SGD
# learning_rate = 1e-2                          # resnet18-AdamW


# Set Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set Augmentations
train_transforms = set_augmentations('train')
valid_transforms = set_augmentations('valid')

# Load Dataset
train_dataset = datasets.ImageFolder("./dataset/train", transform=train_transforms)
valid_dataset = datasets.ImageFolder("./dataset/valid", transform=valid_transforms)

# Set DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
# print(len(train_dataset), '##', len(train_loader))
# exit()

# Call Model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1].out_features = label_volume
model.to(device)
# print(model._get_name())
# exit()

# Set Criterion
criterion = LabelSmoothingCrossEntropy()

# Set Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Set Scheduler
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


if __name__ == '__main__':
    # Train and validate
    train(train_loader, valid_loader, epoch_num, model, device, criterion, optimizer, scheduler)

    # Save last.pt
    torch.save(model.state_dict(), './{}_last.pt'.format(model._get_name()))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


100%|██████████| 3/3 [00:01<00:00,  1.96it/s]
Epoch: [  1 /  50]   Train Loss: 7.189   Train Accuracy: 0.118   Valid Loss: 7.095   Valid Accuracy: 0.103
100%|██████████| 3/3 [00:00<00:00,  3.11it/s]
Epoch: [  2 /  50]   Train Loss: 2.545   Train Accuracy: 0.742   Valid Loss: 3.989   Valid Accuracy: 0.500
100%|██████████| 3/3 [00:00<00:00,  3.30it/s]
Epoch: [  3 /  50]   Train Loss: 2.028   Train Accuracy: 0.840   Valid Loss: 2.614   Valid Accuracy: 0.667
100%|██████████| 3/3 [00:00<00:00,  3.17it/s]
Epoch: [  4 /  50]   Train Loss: 1.697   Train Accuracy: 0.925   Valid Loss: 3.094   Valid Accuracy: 0.564
100%|██████████| 3/3 [00:01<00:00,  2.22it/s]
Epoch: [  5 /  50]   Train Loss: 1.489   Train Accuracy: 0.948   Valid Loss: 2.294   Valid Accuracy: 0.615
100%|██████████| 3/3 [00:00<00:00,  3.41it/s]
Epoch: [  6 /  50]   Train Loss: 1.441   Train Accuracy: 0.951   Valid Loss: 2.085   Valid Accuracy: 0.679
100%|██████████| 3/3 [00:00<00:00,  3.57it/s]
Epoch: [  7 /  50]   Train Loss: 1.3

In [ ]:
test_dataset = datasets.ImageFolder("./dataset/test", transform=valid_transforms)
test_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

test(test_loader, device, label_volume)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can

Accuracy:  88.46153846153845
